### split_csv_by_size.py ->

In [23]:
# -*- coding: utf-8 -*-
"""
staging_all.utf8.csv 를 기반으로:
1) category_dim_seed.csv 생성  (업종 차원 테이블 시드 데이터)
2) merchant_category_map_seed.csv 생성 (가맹점-업종 매핑 시드 데이터)

- category(JSON) 컬럼만 사용
- 업종의 유니크 판정은 (공백 제거 + 소문자 변환 → SHA1 12자리 해시)
- DB에 category_dim 테이블을 먼저 보장 및 업서트(INSERT ... ON DUPLICATE KEY UPDATE)
- category_dim의 ID 매핑을 가져와 merchant_category_map_seed.csv 생성
"""

import csv, json, re, hashlib
from pathlib import Path
from sshtunnel import SSHTunnelForwarder
import pymysql
from tqdm import tqdm

# ===== SSH 접속 정보 =====
SSH_HOST = "j13b108.p.ssafy.io"
SSH_PORT = 22
SSH_USER = "ubuntu"
SSH_KEY = r"C:\Users\SSAFY\Desktop\j13B108T.pem"

# ===== DB 접속 정보 =====
DB_HOST_INSIDE_SSH = "127.0.0.1"   # 터널 내부 MySQL 주소
DB_PORT_INSIDE_SSH = 3306
DB_USER = "admin"
DB_PASS = "dnxlachl"
DB_NAME = "walletslotdb"

# ===== 경로 설정 =====
INFILE = Path("./_merged_csv/staging_all.utf8.csv")   # 입력 CSV (staging_all 통합본)
OUT_DIR = Path("./_prepped")                          # 출력 폴더
OUT_DIR.mkdir(parents=True, exist_ok=True)

OUT_CATEGORY_DIM = OUT_DIR / "category_dim_seed.csv"          # 업종 시드 파일
OUT_MERCHANT_MAP = OUT_DIR / "merchant_category_map_seed.csv" # 가맹점-업종 시드 파일

# ===== 1단계: category_dim_seed.csv 생성 =====
category_set = {}
with open(INFILE, "r", encoding="utf-8") as f:
    reader = csv.DictReader(f)
    for row in reader:
        try:
            # category 컬럼은 JSON 배열 문자열
            category_list = json.loads(row["category"])
            for cat in category_list:
                # 공백 제거 + 소문자로 변환해 유니크 키 생성
                canon = re.sub(r"\s+", "", cat).lower()
                # SHA1 해시의 앞 12자리로 code 생성
                code = hashlib.sha1(canon.encode("utf-8")).hexdigest()[:12]
                # code → 업종명 매핑
                category_set[code] = cat
        except:
            continue

# 업종 시드 CSV 저장
with open(OUT_CATEGORY_DIM, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["name", "code"])
    for code, name in category_set.items():
        writer.writerow([name, code])

# ===== 2단계: SSH 터널 열고 DB 접속 =====
with SSHTunnelForwarder(
    (SSH_HOST, SSH_PORT),
    ssh_username=SSH_USER,
    ssh_pkey=SSH_KEY,
    remote_bind_address=(DB_HOST_INSIDE_SSH, DB_PORT_INSIDE_SSH)
) as tunnel:
    tunnel.start()

    # 로컬 포트 → 원격 MySQL 연결
    conn = pymysql.connect(
        host="127.0.0.1",
        port=tunnel.local_bind_port,
        user=DB_USER,
        password=DB_PASS,
        database=DB_NAME,
        charset="utf8mb4",
        cursorclass=pymysql.cursors.DictCursor,
        autocommit=False
    )

    try:
        with conn.cursor() as cur:

            # ===== 3단계: category_dim 테이블 생성 (없으면 생성) =====
            cur.execute("""
                CREATE TABLE IF NOT EXISTS category_dim (
                    id BIGINT UNSIGNED AUTO_INCREMENT PRIMARY KEY,
                    name VARCHAR(255) NOT NULL,
                    code CHAR(12) NOT NULL,
                    updated_at TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
                    UNIQUE KEY uq_code (code),
                    UNIQUE KEY uq_name (name)
                )
            """)
            conn.commit()

            # ===== 4단계: category_dim 데이터 업서트 =====
            # CSV에 있는 업종을 DB category_dim에 넣고,
            # 중복되면 updated_at만 갱신
            with open(OUT_CATEGORY_DIM, "r", encoding="utf-8") as f:
                next(f)  # header skip
                reader = csv.reader(f)
                for name, code in reader:
                    cur.execute("""
                        INSERT INTO category_dim (name, code)
                        VALUES (%s, %s)
                        ON DUPLICATE KEY UPDATE updated_at = CURRENT_TIMESTAMP
                    """, (name, code))
                conn.commit()

            # ===== 5단계: code → DB id 매핑 =====
            cur.execute("SELECT id, code FROM category_dim")
            code_to_id = {r["code"]: r["id"] for r in cur.fetchall()}

            # ===== 6단계: merchant_category_map_seed.csv 생성 =====
            rows = []
            with open(INFILE, "r", encoding="utf-8") as f:
                reader = csv.DictReader(f)
                for row in reader:
                    try:
                        merchant_code_raw = row["merchant_code"]          # 원본 가맹점 코드
                        merchant_code = merchant_code_raw.strip()        # 정규화
                        source_db = row["source_db"]                     # 데이터 출처
                        category_list = json.loads(row["category"])      # 업종 JSON
                        for cat in category_list:
                            canon = re.sub(r"\s+", "", cat).lower()
                            code = hashlib.sha1(canon.encode("utf-8")).hexdigest()[:12]
                            if code in code_to_id:
                                rows.append([
                                    merchant_code_raw,    # 원본 코드
                                    merchant_code,        # 정규화 코드
                                    code_to_id[code],     # DB에서 할당된 category_dim.id
                                    category_set[code],   # 업종명
                                    source_db             # 데이터 출처
                                ])
                    except:
                        continue

            # 시드 파일로 저장
            with open(OUT_MERCHANT_MAP, "w", newline="", encoding="utf-8") as f:
                writer = csv.writer(f)
                writer.writerow(["merchant_code_raw", "merchant_code", "category_id", "category_name", "source_db"])
                writer.writerows(rows)

            print(f"✅ merchant_category_map_seed.csv 파일 생성 완료 (총 {len(rows)}건)")

    finally:
        conn.close()


✅ merchant_category_map_seed.csv 파일 생성 완료 (총 14873506건)


In [24]:
# -*- coding: utf-8 -*-
"""
merchant_category_map_seed.csv 파일을 5MB 단위로 분할 저장합니다.

- 입력 파일: ./_prepped/merchant_category_map_seed.csv
- 출력 디렉토리: ./_prepped/merchant_category_map_chunks/
- 분할 규칙: 
    * 헤더 1줄은 모든 분할 파일에 공통 포함
    * 본문 데이터는 누적 크기가 5MB를 넘으면 새로운 파일 생성
- 파일명 포맷: merchant_category_map_part_001.csv, merchant_category_map_part_002.csv, ...
"""

import csv
from pathlib import Path

# ===== 경로 설정 =====
INFILE = Path("./_prepped/merchant_category_map_seed.csv")       # 분할 대상 원본 CSV
OUT_DIR = Path("./_prepped/merchant_category_map_chunks")        # 분할된 CSV 저장 폴더
OUT_DIR.mkdir(parents=True, exist_ok=True)                       # 디렉토리 없으면 생성

# ===== 분할 단위 설정 (5MB) =====
CHUNK_SIZE_MB = 5
CHUNK_SIZE_BYTES = CHUNK_SIZE_MB * 1024 * 1024   # 바이트 단위 환산

with open(INFILE, "r", encoding="utf-8") as f:
    header = f.readline()                        # 첫 줄(헤더)은 항상 포함
    part = 1                                     # 분할 파일 번호
    current_chunk = []                           # 현재 파일에 쓸 데이터 줄들
    current_size = len(header.encode("utf-8"))   # 현재 파일 크기 추적 (헤더 포함)

    for line in f:
        encoded_line = line.encode("utf-8")      # 한 줄을 바이트 단위로 계산
        current_chunk.append(line)
        current_size += len(encoded_line)

        # 현재 청크 크기가 기준(5MB) 이상이면 저장
        if current_size >= CHUNK_SIZE_BYTES:
            out_file = OUT_DIR / f"merchant_category_map_part_{part:03}.csv"
            with open(out_file, "w", encoding="utf-8", newline="") as out_f:
                out_f.write(header)              # 헤더 작성
                out_f.writelines(current_chunk)  # 누적된 데이터 작성
            print(f"✅ {out_file.name} 저장 완료")
            part += 1
            current_chunk = []                   # 다음 청크 준비
            current_size = len(header.encode("utf-8"))  # 헤더 크기로 초기화

    # 마지막 남은 데이터도 저장
    if current_chunk:
        out_file = OUT_DIR / f"merchant_category_map_part_{part:03}.csv"
        with open(out_file, "w", encoding="utf-8", newline="") as out_f:
            out_f.write(header)
            out_f.writelines(current_chunk)
        print(f"✅ {out_file.name} 저장 완료 (마지막)")


✅ merchant_category_map_part_001.csv 저장 완료
✅ merchant_category_map_part_002.csv 저장 완료
✅ merchant_category_map_part_003.csv 저장 완료
✅ merchant_category_map_part_004.csv 저장 완료
✅ merchant_category_map_part_005.csv 저장 완료
✅ merchant_category_map_part_006.csv 저장 완료
✅ merchant_category_map_part_007.csv 저장 완료
✅ merchant_category_map_part_008.csv 저장 완료
✅ merchant_category_map_part_009.csv 저장 완료
✅ merchant_category_map_part_010.csv 저장 완료
✅ merchant_category_map_part_011.csv 저장 완료
✅ merchant_category_map_part_012.csv 저장 완료
✅ merchant_category_map_part_013.csv 저장 완료
✅ merchant_category_map_part_014.csv 저장 완료
✅ merchant_category_map_part_015.csv 저장 완료
✅ merchant_category_map_part_016.csv 저장 완료
✅ merchant_category_map_part_017.csv 저장 완료
✅ merchant_category_map_part_018.csv 저장 완료
✅ merchant_category_map_part_019.csv 저장 완료
✅ merchant_category_map_part_020.csv 저장 완료
✅ merchant_category_map_part_021.csv 저장 완료
✅ merchant_category_map_part_022.csv 저장 완료
✅ merchant_category_map_part_023.csv 저장 완료
✅ merchant_

In [25]:
# -*- coding: utf-8 -*-
"""
merchant_category_map_part*.csv 파일들을 MySQL DB에 적재하는 스크립트

- SSH 터널을 통해 원격 MySQL 접속
- CSV 데이터를 읽어 INSERT IGNORE 방식으로 삽입
- 배치 단위로 Multi-VALUES INSERT 실행 → 성능 최적화
- 각 CSV 파일은 1회 커밋 (파일 단위 트랜잭션)
- 처리 완료된 파일은 .done 마커 파일 생성하여 재실행 방지
"""

import os, time, csv
from pathlib import Path
from typing import Iterator, List, Tuple
from sshtunnel import SSHTunnelForwarder
import pymysql

# ===== SSH 설정 =====
SSH_HOST = "j13b108.p.ssafy.io"                          # 원격 SSH 호스트
SSH_PORT = 22                                            # SSH 포트
SSH_USER = "ubuntu"                                      # SSH 사용자
SSH_KEY  = r"C:\Users\SSAFY\Desktop\j13B108T.pem"        # SSH 키 파일 경로

# ===== MySQL 설정 (터널 내부 주소) =====
DB_HOST_INSIDE_SSH = "127.0.0.1"                         # MySQL 서버 주소 (터널 내부 기준)
DB_PORT_INSIDE_SSH = 3306                                # MySQL 포트
DB_USER = "admin"                                        # MySQL 사용자
DB_PASS = "dnxlachl"                                     # MySQL 비밀번호
DB_NAME = "walletslotdb"                                 # DB 이름

# ===== CSV 경로 =====
CHUNK_DIR = Path("./_prepped/merchant_category_map_chunks")  # 분할된 CSV 파일 저장 경로
PATTERN = "merchant_category_map*.csv"                       # 대상 CSV 파일 패턴
csv_files = sorted(f for f in CHUNK_DIR.glob(PATTERN))       # 대상 CSV 목록

# ===== 테이블 생성 SQL =====
CREATE_TABLE_SQL = """
CREATE TABLE IF NOT EXISTS merchant_category_map (
  id BIGINT UNSIGNED AUTO_INCREMENT PRIMARY KEY,
  merchant_code VARCHAR(100) NOT NULL,
  category_id BIGINT UNSIGNED NOT NULL,
  created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
  UNIQUE KEY uq_merchant_category (merchant_code, category_id),
  FOREIGN KEY (category_id) REFERENCES category_dim(id),
  FOREIGN KEY (merchant_code) REFERENCES staging_merchants(merchant_code)
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;
"""

# ===== Insert SQL Prefix & Placeholder =====
INSERT_PREFIX = """
INSERT IGNORE INTO merchant_category_map
(merchant_code, category_id, created_at)
VALUES
"""
ROW_PH = "(%s, %s, CURRENT_TIMESTAMP)"   # 한 로우에 대한 플레이스홀더


# ===== CSV 로우 파싱 함수 =====
def iter_rows_from_csv(fname: Path) -> Iterator[Tuple[str, int]]:
    """
    CSV 파일을 열어 (merchant_code, category_id) 튜플 생성
    """
    with open(fname, encoding="utf-8") as f:
        for row in csv.DictReader(f):
            merchant_code = row.get("merchant_code")
            category_id = row.get("category_id")
            if not merchant_code or not category_id:  # 값이 없으면 스킵
                continue
            yield (merchant_code, int(category_id))


# ===== Multi-Row Insert 실행 =====
def exec_batch_multi_values(cursor, rows: List[Tuple], batch_size=5000) -> int:
    """
    rows 리스트를 batch_size 단위로 나눠 Multi-VALUES Insert 실행
    - cursor.rowcount 로 실제 삽입된 로우 수 집계
    """
    total = 0
    for i in range(0, len(rows), batch_size):
        batch = rows[i:i+batch_size]
        phs = ",\n".join([ROW_PH] * len(batch))       # 플레이스홀더 반복
        flat_vals = [val for row in batch for val in row]  # 2차원 → 1차원 flatten
        sql = INSERT_PREFIX + phs
        cursor.execute(sql, flat_vals)
        total += cursor.rowcount
    return total


# ===== 개별 파일 처리 =====
def load_one_file(cur, fn: Path) -> Tuple[int, int, float]:
    """
    한 CSV 파일 처리:
    - CSV 로우 읽기
    - DB에 Multi-VALUES Insert
    - 실행 시간 반환
    """
    t0 = time.time()
    rows = list(iter_rows_from_csv(fn))      # 모든 로우 읽기
    n_rows = len(rows)                       # 읽은 로우 수
    n_ins = exec_batch_multi_values(cur, rows)  # 삽입된 로우 수
    secs = time.time() - t0                  # 실행 시간
    return n_rows, n_ins, secs


# ===== 메인 실행 =====
def main():
    t_all = time.time()
    total_rows = total_ins = 0

    # SSH 터널 연결
    with SSHTunnelForwarder(
        (SSH_HOST, SSH_PORT),
        ssh_username=SSH_USER,
        ssh_pkey=SSH_KEY,
        remote_bind_address=(DB_HOST_INSIDE_SSH, DB_PORT_INSIDE_SSH)
    ) as tunnel:
        tunnel.start()
        local_port = tunnel.local_bind_port
        print(f"[SSH] {tunnel.local_bind_host}:{local_port} -> {DB_HOST_INSIDE_SSH}:{DB_PORT_INSIDE_SSH}")

        # MySQL 연결
        conn = pymysql.connect(
            host="127.0.0.1",
            port=local_port,
            user=DB_USER,
            password=DB_PASS,
            database=DB_NAME,
            charset="utf8mb4",
            autocommit=False,  # 파일 단위 커밋
            read_timeout=600,
            write_timeout=600,
        )

        with conn.cursor() as cur:
            # 테이블 보장
            cur.execute(CREATE_TABLE_SQL)

            # CSV 파일 순회
            for fn in csv_files:
                done_marker = fn.with_suffix(".done")
                if done_marker.exists():  # 이미 처리한 파일은 스킵
                    continue
                n_rows, n_ins, secs = load_one_file(cur, fn)
                conn.commit()  # 파일 단위 커밋
                total_rows += n_rows
                total_ins += n_ins
                print(f"[DONE] {fn.name:>28s} | 읽음 {n_rows:>8,} / 삽입 {n_ins:>8,} | {secs:5.1f}s | OK")
                done_marker.touch()  # 마커 파일 생성

        conn.close()
        tunnel.close()

    # 전체 요약 출력
    print(f"\n✅ 전체 합계: 읽음 {total_rows:,} / 삽입 {total_ins:,} | 총 {time.time() - t_all:.1f}s")


if __name__ == "__main__":
    main()


[SSH] 0.0.0.0:50540 -> 127.0.0.1:3306

✅ 전체 합계: 읽음 0 / 삽입 0 | 총 0.9s
